# 케라스로 신경망 작성하기

케라스의 신경망 학습은 크게 두단계로 이뤄진다. 

- 모델 생성
- 모델 학습 / 검증

모델이란 신경망의 구성을 의미한다. 일반적인 입력층, 은닉층, 출력층등의 각각의 계층과 이것을 학습하기 위한 파라미터등이 이 모델에 포함된다. 

<hr style="height:2px" > 

## 가장 심플한 신경망

다음은 가장 간단한 2x2 신경망의 예를 하나 생성해 보았다. 


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#keras.backend.clear_session()
model = keras.models.Sequential()
model.add(layers.Dense(2, activation = 'sigmoid', input_shape = (2,))) 
model.add(layers.Dense(2, activation = 'softmax'))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc', 'mae'])      


다음은 이를 학습하는 예이다. 

In [0]:
import numpy as np
x = np.array([[0,1],[1,0]]).astype("float32")
y = np.array([[1,0],[0,1]]).astype("float32")
hist = model.fit(x,y, epochs = 100, batch_size = 3)

이제 위의 예제를 하나씩 분석해서 신경망 학습의 절차를 보도록 하자. 

<hr>

# 모델 생성

케라스의 모델 생성은 기본적으로 다음과 같은 과정을 거친다. 

1. 모델을 선언
2. 모델에 레이어를 추가 
3. 모델 컴파일


### keras 라이브러리 불러오기

layer는 여러번 쓰게 되므로 keras.layers  를 쓰기 귀찮아서 직접 임포트하였다. 

In [0]:
import keras 
from keras import layers

### 간단한 순차 모델 선언

신경망의 가장 기본이 되는 구조는 순차적으로 레이어를 배치하고 역전파 모델로 이를 학습하는 방법이다. 사실 현대에는 다양한 파생구조가 존재하지만 우선은 이러한 순차모델부터 익혀보자. 

In [0]:
model = keras.models.Sequential() 

이제  model 이라는 변수가 신경망을 의미하게된다.

### 입력 레이어 추가 

가장 간단한 모델로 입력값 두개를 받는 입력 레이어를 생성한다. 이는 model.add 를 이용해 층을 추가함으로서 이루어진다. 

가장 초기부터 널리 사용되는 층은 입력값을 가중치를 곱해 합산후에 필터를 거치는 방식이다. 이러한 층을 케라스에서는 
layers.Dense 로 정의한다. Dense 계층은 기본적으로 node 숫자를 요구한다. 

    layers.Dense([노드숫자],.. 옵션들)

단 중간계층은 전층의 출력에 의해 자동으로 입력이 주어지지만 입력계층은 입력값의 형태를 같이 옵션으로 넣어줘야한다. 이는 다음과 같은 옵션을 사용한다. 아래의 옵션은 2개의 값으로 이뤄진 1차원 배열 형태로 입력값을 끊어서 입력받음을 의미한다. 

    input_shape = (2,)

activation 옵션은 합산된 입력값을 다시 출력값으로 전환하는 과정에서 사용하는 함수로서 이를 활성함수라고 한다. 가장 유명한 함수로 sigmoid 가 있지만 속도등의 이유로 relu 가 기본값으로 입력되어있다. 

    activation = 'relu' 

이제 위의 예에서 사용된 입력 계층을 한번 살펴보자.     

In [0]:
model.add(layers.Dense(2, activation = 'sigmoid', input_shape = (2,))) 

### 출력 레이어 추가 

출력이나 은닉레이어는 자동으로 input_shape 가 결정되므로 input_shape 옵션은 필요로 하지 안는다. 대신 활성화 함수로서 softmax 를 주로 사용하는데 softmax 는 출력값에서 강한 출력값을 더 강화해주고 작은 출력값은 더 작게 만들어준다. 예를 들어 [0.2, 0.7] 을 [0.1, 0.9] 정도로 차이를 강화하는 효과가 있다. 

위의 예에서는 2x2 의 신경망이므로 다음과 같이 출력층을 정의하였다. 

In [0]:
model.add(layers.Dense(2, activation = 'softmax'))

### 모델 컴파일 

모델을 컴파일하는 이유는 정의된 신경망의 학습 옵션을을 정의하고, 가중치의 초기화를 수행하기 위해서이다.이 모델 컴파일에 필수적인 옵션 요소는 다음과 같다. 

    optimizer : 가중치를 업데이트하는 방식으로 경사하강법(sgd),adam, rmsprop 등등이 있음
    loss : 오차를 계산하는 방식 'categorical_crossentropy' ,'squared_mean_error' 등등이 있음
    metircs : loss 이외에 성능 평가를 위한 지표 'acc', 'mae'  등이 있음
  
metrics 는 여러 지표를 선택하기 위해 배열형태로 입력하여야한다. (예: metrics = ['acc', 'mae'] )
    


In [0]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc'])

### 모델 요약 및 초기화 

생성된 모델 정보를 검색하기 위해서는 다음과 같은 함수를 호출하면 된다. 



In [0]:
model.summary() 

그런데 주피터에서 작업을 많이 하다보면 모델생성코드를 재실행시 dense 값들이 계속 올라가는 것을 볼 수 있을것이다. 이는 keras 의 밑단이 되는 tensorflow 구조가 계속 누적된다는 의미이며 이것이 학습에 영향을 주는 경우도 있기 때문에 이를 초기화시키려면 다음과 같은 코드를 실행하면 된다. 

In [0]:
keras.backend.clear_session()


<hr> 

# 데이터 입력 및 학습 실행

데이터의 학습은 생각보다 간단하다. model.fit 함수에 입력, 예상 출력 값을 넣어주고 반복시행시 필요한 옵션을 입력하면 된다. 

    model.fit( [x], [y], 반복 옵션.. ) 
    

### 입력데이터(x)의 생성

대부분 파이썬 학습이 그러하듯 케라스도 numpy 데이터를 입력으로 사용한다. 그리고 역시 대부분이 그러하듯 표준데이터는 다음과 같이 전체케이스를  2차원 배열로 한방에 넣는 것을 선호한다. 

만일 다음과 같은 케이스가 있다고 가정하자. 

    x : [0,1]  => y : 0
    x : [1,0]  => y : 1 

이때의 입력데이터 x 는 다음과 같이 정의하면 된다. 

    np.array( [[0,1], [1,0]] ) 
  
만일 입력값이 3개라면 [[0,0,1], [0,1,0], .. ] 형태가 된다. 그리고 대부분의 x 데이터는 실수값이어야 하므로 최종적으로 x 의 형태는 다음과 같다. 

In [0]:
x = np.array( [[0,1], [1,0]] ).astype('float32')

   
### 출력 기대값(y)의 생성

분류(classification) 문제의 경우 출력 값은 해당 클래스를 의미하 경우가 많다.  이런 경우 y를 범주형으로 간주하는데 이 경우는 y값이 0과 1으로 표현되었지만 이를 각각 두개의 서로 다른 노드의 출력으로 유도하기 위해 [0,1] [1,0] 으로 표현한다. 이 역시 되도록이면 실수로 표현하는게 좋으므로 다음과 같이 표현한다. 
       

In [0]:
y = np.array( [[0,1], [1,0]] ).astype('float32')

### 학습 실행 

이제 학습을 위해 model.fit을 실행하기 위한 데이터의 준비가 끝났다. 단 실행시 두가지 반복옵션이 필요하다. 

    epochs : 전체 반복횟수
    batch_size : 한번 반복시 읽어들이는 데이터량 
    
batch_size 는 병렬로 처리되는 케이스 크기로서 작은 입력케이스의 경우 케이스 전체숫자를 한기준으로 하는 경우가 많다. 실제 학습횟수는 케이스의 양은 batch_size * epochs 가 된다.  위의 예제에서는 케이스가 2개이므로 batch_size를 2로 설정하였다.    
    

In [0]:
hist = model.fit(x,y, epochs = 1000, batch_size = 2)

### <font color = 'red'> 연습문제 1-1 : 다음의 데이터를 학습해보세요
    
    x : 0,0,1 -> 1
    x : 1,0,0 -> 2
    x : 0,1,0 -> 3
    
    

In [0]:
## 연습문제 1-1 코드를 여기서 작성해보세요 

<hr style="height:2px" > 

# 은닉층과 xor 문제의 해결

xor 문제는 다층신경망 - 역전파 모델을 탄생시킨 모티브가 된 유명한 문제이다. 이는 입력 x 출력 으로 구성되는 단층 신경망으로는 해결이 불가능하며 이를 위해 다층 신경망이 필요하다. 

이번에는 먼저 문제의 데이터를 정의히자. 

###  xor문제의 입력데이터 

In [0]:
import numpy as np

x = np.array([[0,0], [0,1], [1,0], [1,1]]).astype("float32")
y = np.array([[0,1], [1,0], [1,0], [0,1]]).astype("float32")


### 은닉층을 포함한 신경망 구성 

In [0]:
from keras import models
from keras import layers
keras.backend.clear_session
model = models.Sequential() 
model.add(layers.Dense(2, activation = 'sigmoid', input_shape = (2,)))
model.add(layers.Dense(4))
model.add(layers.Dense(2, activation ='softmax'))
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc'])

### 학습 실행 

xor는 보기보다 난이도가 있는 문제이다. 그러므로 생각보다 많은 epoch를 요구한다. 

In [0]:
model.fit(x,y, epochs = 1000, batch_size = 4)

### 그래프로 모니터링 하기 

반복횟수(epochs)가 1000회가 넘어가면 사실 이 학습이 제대로 이뤄지고 있는지 확인하는 것도 만만한 일이 아니다. 그리고 눈으로 스크롤하며 변화치를 봐서는 이게 제대로 학습이 이루어지고 있는지 확인하기도 어렵다. 이제 이 loss 변화를 직접 눈으로 확인할 수 있는 방법을 알아보자. 

model.fit 은 각 epoch 당 학습의 결과값들을 history형태로 리턴한다. 간단하게 history 의 형태를 보자. 



In [0]:
hist = model.fit(x,y, epochs = 10, batch_size = 40)
print( hist.history )

tuple 형태로 'loss' 와 'acc' 가 전달된 것을 볼 수 있을 것이다. 이제 이것을 그래프로 표현해보도록 하겠다. 


In [0]:
import matplotlib.pyplot as plt 
plt.plot(hist.history['loss'])
plt.show()

이제 학습이 이뤄진 loss 의 변화 과정을 그래프로 한눈에 볼수 있게 되었다. 그런데 학습 결과를 아래로 한참 스크롤 하여 그래프 결과를 보는 것도 상당히 귀찮은 일이다.  이를 위해서 학습 과정시 나오는 바와, 수치 출력을 제거할 수 있는 옵션이 있다. 

    model.fit( ... verbose = 0 )
 
이제 위의 xor 학습을 메시지 없이 그래프 형태로 모니터링하도록 바꿔보자. 


In [0]:
import matplotlib.pyplot as plt 
hist = model.fit(x,y, epochs = 1000, batch_size = 4, verbose  = 0)
plt.plot(hist.history['loss'])
plt.show()
print("last acc :", hist.history['acc'][-1])


 <font color = 'red'> 
### 연습문제 1-2 : xor epoch 최소화 
acc 가  1.0 에 도달한 최초 epoch 를 찾고 batch_size 를 바꾸며 이를 최소화할 수 있게 실행해보자. 
    
    
    

In [0]:
## 연습문제 1-2 코드를 여기에 작성해서 돌려보세요 
import matplotlib.pyplot as plt 
hist = model.fit(x,y, epochs = 4000, batch_size = 4, verbose  = 0)
plt.plot(hist.history['loss'])
plt.show()
print("last acc :", hist.history['acc'][-1])
print( len( hist.history['acc']) )

<hr style="height:2px" > 

#  학습 모델 공간 분포 시각화 

### 학습 결과 활용 하기 

신경망으로 생성된 분류기는 실제 데이터를 판별하기 위해 적용될 수 있다. 학습된 망에 직접 커스텀 데이터를 넣어서 확인하는 방법은 다음과 같다. 


In [0]:
print(model.predict( np.array([[0,1]])))

학습은 0과 1로 이뤄졌지만 신경망은 수학적 실수연산의 네트워크이므로 그 중간값에 대해서도 출력이 가능하다. 당연히 0.5,0.5 도 그 결과값을 리턴한다. 

In [0]:
print(model.predict( np.array([[0.5,0.5]])))

그렇다면 신경망이 학습한 상태공간은 어떻게 구성되어있을까? 직접 보고싶은 생각이 들것이다.  이는 간단하다. x,y 0부터 1까지의 공간의 점을 모두 신경망에 넣어보면 된다. 

물론 평면의 점의 갯수는 무한개지만 일정 간격으로 점을 추출하면 유한개의 점이 추출된다. 하지만 이런 점들의 집합을 어떻게 생성할까? 물론 이중 루프를 돌려서 생성하는 방법도 있다. 하지만 numpy는 좀더 간단한 방법으로 meshgrid 라는 방법을 제공한다. 


In [0]:
import matplotlib.pyplot as plt 

x = np.mgrid[0:1:0.1, 0:1:0.1].reshape(2,-1).T
print(x)
plt.scatter(x[:,0], x[:,1])
plt.show()

이번에는 x를 model에 입력한 결과값을 한번 보자. 

In [0]:
y = model.predict(x)
print(y)

위의 결과는 단순한 신경망의 출력값이다. 물론 이 값에서 앞의 값이 더 크면 0, 뒤의 값이 더 크면 1로 간주해도 좋을 것이다. 그렇지만 이것을 시각화하려면 0,1.. 등의 값이 필요하다. 어떻게 해야할까?  

물론 모든것이 그러하듯 하드코딩해도 상관은 없을 테지만.. 코드가 복잡해지는걸 막기위해서 여기서는 해 배열중에 가장 큰 배열의 인덱스 값을 출력하는 numpy 함수인 argmax 를 이용하도록 하겠다. 단 가로축(axis=1) 기준으로 큰 값의 인덱스를 생성하도록 다음과 같이 사용한다. 

In [0]:
print(np.argmax(y, axis = 1 ))

이제 준비가 다 되었다. xor 신경망이 생성한 상태공간의 모습을 직접 보는 전체 코드를 보자. 

In [0]:
import matplotlib.pyplot as plt 

x = np.mgrid[0:1:0.02, 0:1:0.02].reshape(2,-1).T
y = model.predict(x)
plt.scatter(x[:,0], x[:,1], marker = '.', c = np.argmax(y, axis = 1))
plt.show()

 <font color = 'red'> 
### 연습문제 1-3 : 신경망으로 그림 그리기 
신경망의 학습 샘플을 생성해 다양한 형태의 상태공간모양을 만들어보자. 
    
    
    